<a href="https://colab.research.google.com/github/LeonardoLuca/AskYou/blob/main/rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste conexão Gemini


In [ ]:
# Célula 1: Instalar as bibliotecas necessárias
!pip install -q langchain langchain-google-genai datasets qdrant-client tiktoken


In [ ]:
# Célula 2: Importar bibliotecas e configurar a API Key do Google
import os
import sys
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

# Variável para verificar se a configuração foi bem sucedida
google_api_key_configurada = False
chat = None # Inicializa a variável chat

try:
    # Pega a chave de API dos Secrets do Colab
    # Certifique-se que o nome do secret é 'GOOGLE_API_KEY'
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    # Verifica se a chave foi realmente obtida (não é None ou vazia)
    if not GOOGLE_API_KEY:
        print("Erro: A chave de API 'GOOGLE_API_KEY' foi encontrada nos Secrets, mas está vazia.", file=sys.stderr)
    else:
        print("Chave de API do Google carregada com sucesso!")
        google_api_key_configurada = True

except userdata.SecretNotFoundError:
    print("Erro: Secret 'GOOGLE_API_KEY' não encontrado.", file=sys.stderr)
    print("Por favor, adicione sua chave de API do Google AI Studio aos Secrets do Colab.", file=sys.stderr)
except Exception as e:
    print(f"Ocorreu um erro inesperado ao buscar a chave de API: {e}", file=sys.stderr)

# Limpa a variável do ambiente se ela existir por algum motivo (boa prática)
# Embora estejamos passando diretamente, é bom garantir que não haja conflito.
if 'GOOGLE_API_KEY' in os.environ:
    del os.environ['GOOGLE_API_KEY']

In [ ]:
# Célula 3: Instanciar o modelo Gemini (só executa se a chave foi carregada)

if google_api_key_configurada:
    try:
        # Escolha o modelo Gemini (ex: 'gemini-1.5-pro-latest', 'gemini-1.5-flash-latest')
        chat = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro-latest",  # Ou outro modelo Gemini compatível
            google_api_key=GOOGLE_API_KEY,
            # convert_system_message_to_human=True # Descomente se tiver problemas com SystemMessage
                                                 # Isso faz com que a mensagem do sistema seja anexada
                                                 # à primeira mensagem humana, o que às vezes funciona melhor com Gemini.
        )
        print(f"Modelo {chat.model} instanciado com sucesso.")
    except Exception as e:
        print(f"Erro ao instanciar o modelo ChatGoogleGenerativeAI: {e}", file=sys.stderr)
        chat = None # Garante que chat é None se a instanciação falhar
else:
    print("\nInstanciação do modelo pulada pois a chave de API não foi configurada.", file=sys.stderr)

In [ ]:
# Célula 4: Definir as mensagens (geralmente sem alterações)
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="Você é um assistente útil que responde perguntas."),
    HumanMessage(content="Olá Bot, como você está hoje?"),
    AIMessage(content="Estou bem, obrigado por perguntar! Como posso ajudar?"),
    HumanMessage(content="Gostaria de entender o que é Machine Learning")
]

print("Lista de mensagens definida.")

In [ ]:
# Célula 5: Invocar o modelo e obter a resposta (com verificação)

res = None # Inicializa a variável de resultado

if chat: # Verifica se o modelo foi instanciado com sucesso
    try:
        print(f"\nEnviando mensagens para o modelo {chat.model}...")
        res = chat.invoke(messages)
        print("Resposta recebida do modelo.")
    except Exception as e:
        print(f"Erro durante a invocação do modelo: {e}", file=sys.stderr)
else:
    print("\nInvocação pulada pois o modelo não foi instanciado corretamente (verifique a API Key e a Célula 3).", file=sys.stderr)

# Exibe o objeto de resposta completo (opcional, útil para debug)
# print("\nObjeto de resposta completo:")
# print(res)

In [ ]:
# Célula 6: Imprimir o conteúdo da resposta (com verificação)

if res: # Verifica se houve uma resposta válida
    print("\n--- Conteúdo da Resposta ---")
    print(res.content)
    print("--------------------------")
elif chat: # Se o chat foi instanciado mas res é None, houve erro na invocação
     print("\nNão foi possível obter uma resposta do modelo (verifique o erro na Célula 5).", file=sys.stderr)
else:
     print("\nNão há resposta para exibir pois o modelo não foi instanciado.", file=sys.stderr)

In [ ]:
messages.append(res)

In [ ]:
messages

In [ ]:
prompt = HumanMessage (
    content="Qual a diferença entre supervisionado e não supervisionado?"
    )
messages.append(prompt)

In [ ]:
res = chat.invoke(messages)
print(res.content)

# RAG

In [ ]:
#Conectando com o google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
#Importando o dataset do drive
import json

# Path to your JSON file in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/datasets/dataset-cv-leonardo.json'

# Load the JSON file
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Display the first entry to verify
print(dataset[0])

In [ ]:
#Bibliotecas para embedding
!pip install -q sentence-transformers faiss-cpu transformers torch

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Extract descriptions from the dataset
descriptions = [entry['Description'] for entry in dataset]

# Generate embeddings
embeddings = embedder.encode(descriptions, show_progress_bar=True)

# Convert to numpy array for FAISS
embeddings = np.array(embeddings)

In [ ]:
print(descriptions[0])
print(embeddings)

In [ ]:
import faiss

# Create a FAISS index
dimension = embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity

# Add embeddings to the index
index.add(embeddings)

# Save the index (optional)
faiss.write_index(index, '/content/drive/My Drive/Colab Notebooks/index/faiss-cv-leonardo-index.bin')

In [ ]:
#Load a Generative Model
from transformers import pipeline

# Load a generative model. Utilizando modelo do chat gpt, que não estava comaptivel com o primeiro exemplo.
#Foi necessário alterar o código para integrar com o exemplo do GEMINI
generator = pipeline('text-generation', model='distilgpt2')

ModuleNotFoundError: No module named 'transformers'

In [ ]:
#Implement the RAG Pipeline

def rag_query(query, k=3):
    # Encode the query
    query_embedding = embedder.encode([query])

    # Search for the top-k most similar entries
    distances, indices = index.search(np.array(query_embedding), k)

    # Retrieve corresponding dataset entries
    retrieved_entries = [dataset[idx] for idx in indices[0]]

    # Create a context from retrieved entries
    context = "\n".join([entry['Description'] for entry in retrieved_entries])

    # Generate a response
    prompt = f"Baseado nessa informação a seguir:\n{context}\n\nResponda a questão: {query}"
    response = generator(prompt, max_length=200, num_return_sequences=1)[0]['generated_text']

    return response, retrieved_entries

# Example query
query = "Qual a experiência de Leonardo com Python?"
response, retrieved = rag_query(query)
print("Respostas:", response)
print("Retrieved Entries:", retrieved)